<a href="https://colab.research.google.com/github/jkauffm4/Intro-to-ML/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import torch

Mounted at /content/drive


In [2]:
def model(t_u, w2, w1, b):
  return w2 * t_u ** 2 + w1 * t_u + b

In [3]:
def loss_fn(t_p, t_c):
  squared_diffs = (t_p-t_c)**2
  return squared_diffs.mean()

In [4]:
t_c = [.05, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 50.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [5]:
n_samples = t_u.shape[0]
n_val = int(0.2* n_samples)
shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]
val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

In [71]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
  for epoch in range(1, n_epochs +1):
    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model(train_t_u, *params)
    train_loss = loss_fn(train_t_p, train_t_c)

    with torch.no_grad():
      val_t_p = model(val_t_u, *params)
      val_loss = loss_fn(val_t_p, val_t_c)
      assert val_loss.requires_grad == False

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
            f" Validation loss {val_loss.item():.4f}")

  return params

In [60]:
params = torch.tensor([1.0, 1.0, 1.0], requires_grad=True)
learning_rate = 1e-1
optimizer = torch.optim.Adam([params], lr=learning_rate)

training_loop(n_epochs = 5000,
              optimizer = optimizer,
              params = params,
              train_t_u = train_t_u,
              val_t_u = val_t_u,
              train_t_c = train_t_c,
              val_t_c = val_t_c
              )

Epoch 500, Training loss 8.3715, Validation loss 0.7178
Epoch 1000, Training loss 5.7811, Validation loss 7.3991
Epoch 1500, Training loss 4.4338, Validation loss 21.8965
Epoch 2000, Training loss 4.0155, Validation loss 35.2919
Epoch 2500, Training loss 3.9306, Validation loss 42.4976
Epoch 3000, Training loss 3.9055, Validation loss 44.6940
Epoch 3500, Training loss 3.8811, Validation loss 44.7062
Epoch 4000, Training loss 3.8509, Validation loss 44.0736
Epoch 4500, Training loss 3.8133, Validation loss 43.1975
Epoch 5000, Training loss 3.7672, Validation loss 42.1052


tensor([ 0.0084, -0.2170, -1.6883], requires_grad=True)

2, Linear Regression model

In [59]:
lin_model = torch.nn.Linear(5,1) #5 inputs + bias to 1 output linear regression model

In [69]:
def linear_training_loop(n_epochs, optimizer, model, loss_fn, train_x, val_x, train_y, val_y):
  for epoch in range(1, n_epochs + 1):

    pred_y = model(train_x)
    train_loss = loss_fn(pred_y, train_y)

    with torch.no_grad():
      val_pred_y = model(val_x)
      val_loss = loss_fn(val_pred_y, val_y)
      assert val_loss.requires_grad == False

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
            f" Validation loss {val_loss.item():.4f}")

  return model.weight, model.bias

In [61]:
#varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
def binary_map(x):
  return x.map({'yes': 1, 'no': 0})
#test = binary_map(varlist)

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
file_path = '/content/drive/My Drive/Intro-to-ML/Housing.csv'
sample = pd.DataFrame(pd.read_csv(file_path))
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
sample[varlist] = sample[varlist].apply(binary_map)
np.random.seed(0)
train_set, test_set = train_test_split(sample, train_size = .8, test_size = .2, random_state = 100)
Y = train_set.pop('price')
Y_test = test_set.pop('price')
junk = train_set.pop('furnishingstatus') #Gotta remove this, its not used
junk = test_set.pop('furnishingstatus') #Same with this
blerg = train_set
train_set_all = train_set.to_numpy()
test_set_all = test_set.to_numpy()
# Theres a better way to get out the values here, but I don't know it
junk = train_set.pop('mainroad')
junk = train_set.pop('guestroom')
junk = train_set.pop('hotwaterheating')
junk = train_set.pop('basement')
junk = train_set.pop('airconditioning')
junk = train_set.pop('prefarea')
junk = test_set.pop('mainroad')
junk = test_set.pop('guestroom')
junk = test_set.pop('hotwaterheating')
junk = test_set.pop('basement')
junk = test_set.pop('airconditioning')
junk = test_set.pop('prefarea')
X_train = train_set.to_numpy()
X_test = test_set.to_numpy()
Y = Y.to_numpy()
Y_test = Y_test.to_numpy()
size = len(X_train)
x0 = np.ones((size,1))

#Dont think this part is needed because the linear model adds bias on its own
#X_new_train = np.concatenate((x0,X_train), axis=1)
#x0 = np.ones((len(X_test),1))
#X_new_test = np.hstack((x0, X_test))

In [63]:
# Standardize all your stuff
X_standard_test = StandardScaler().fit_transform(X_test)
X_standard_train = StandardScaler().fit_transform(X_train)
train_x = torch.tensor(X_standard_train, dtype=torch.float32)
train_y = torch.tensor(Y, dtype=torch.float32)
val_x = torch.tensor(X_standard_test, dtype=torch.float32)
val_y = torch.tensor(Y_test, dtype=torch.float32)

In [70]:
optimizer = torch.optim.Adam(lin_model.parameters(), lr=1e-4)

linear_training_loop(n_epochs = 5000,
              optimizer = optimizer,
              model = lin_model,
              loss_fn = loss_fn,
              train_x = train_x,
              train_y = train_y,
              val_x = val_x,
              val_y = val_y)

Epoch 500, Training loss 26477594673152.0000, Validation loss 25158758694912.0000
Epoch 1000, Training loss 26477594673152.0000, Validation loss 25158758694912.0000
Epoch 1500, Training loss 26477594673152.0000, Validation loss 25158758694912.0000
Epoch 2000, Training loss 26477594673152.0000, Validation loss 25158758694912.0000
Epoch 2500, Training loss 26477592576000.0000, Validation loss 25158758694912.0000
Epoch 3000, Training loss 26477592576000.0000, Validation loss 25158758694912.0000
Epoch 3500, Training loss 26477592576000.0000, Validation loss 25158758694912.0000
Epoch 4000, Training loss 26477592576000.0000, Validation loss 25158758694912.0000
Epoch 4500, Training loss 26477592576000.0000, Validation loss 25158754500608.0000
Epoch 5000, Training loss 26477592576000.0000, Validation loss 25158754500608.0000


(Parameter containing:
 tensor([[ 0.0204,  0.0910,  0.1928, -0.0199, -0.0062]], requires_grad=True),
 Parameter containing:
 tensor([0.8759], requires_grad=True))

Question 3

In [19]:
x_train_all = torch.tensor(train_set_all, dtype=torch.float32)
y_train_all = torch.tensor(Y, dtype=torch.float32)
x_test_all = torch.tensor(test_set_all, dtype=torch.float32)
y_test_all = torch.tensor(Y_test, dtype=torch.float32)

In [47]:
lin_model_all = torch.nn.Linear(11,1)

In [55]:
optimizer = torch.optim.Adam(lin_model_all.parameters(), lr=1e-1)

linear_training_loop(n_epochs = 5000,
              optimizer = optimizer,
              model = lin_model_all,
              loss_fn = loss_fn,
              train_x = x_train_all,
              train_y = y_train_all,
              val_x = x_test_all,
              val_y = y_test_all)

Epoch 500, Training loss 6835577815040.0000, Validation loss 6943963873280.0000
Epoch 1000, Training loss 6835048284160.0000, Validation loss 6943445876736.0000
Epoch 1500, Training loss 6834518228992.0000, Validation loss 6942926307328.0000
Epoch 2000, Training loss 6833988173824.0000, Validation loss 6942407262208.0000
Epoch 2500, Training loss 6833459691520.0000, Validation loss 6941891887104.0000
Epoch 3000, Training loss 6832929636352.0000, Validation loss 6941371269120.0000
Epoch 3500, Training loss 6832399581184.0000, Validation loss 6940850651136.0000
Epoch 4000, Training loss 6831869526016.0000, Validation loss 6940338421760.0000
Epoch 4500, Training loss 6831339995136.0000, Validation loss 6939815706624.0000
Epoch 5000, Training loss 6830811512832.0000, Validation loss 6939289321472.0000


(Parameter containing:
 tensor([[ 783.8562, 5906.1865, 5889.3516, 5901.1035, 5877.9204, 5684.1172,
          5879.0708, 5904.3730, 5740.9941, 5442.5708, 5459.4810]],
        requires_grad=True),
 Parameter containing:
 tensor([5914.8076], requires_grad=True))